In [1]:
import osmnx as ox
from streamlit_functions import get_nodes_with_tags_in_bbox, count_tag_frequency, gdf_data

In [55]:
places = ["Mitte, Berlin"]
places_gdf = ox.geocode_to_gdf(places)
# bbox = [S, W, N, E]
bounding_boxes = places_gdf.loc[:, ["bbox_south", "bbox_west", "bbox_north", "bbox_east",]]
places_gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((13.36586 52.53566, 13.36620 52.53533...",52.540401,52.504037,13.429402,13.36586,307479808,relation,16566,52.517885,13.40406,"Mitte, Berlin, Germany",boundary,administrative,0.733969


In [69]:
import pandas as pd

ping_pong = {'version': 0.6, 'generator': 'Overpass API 0.7.60.6 e2dc3e5b', 'osm3s': {'timestamp_osm_base': '2023-06-29T15:35:14Z', 'timestamp_areas_base': '2023-06-29T12:13:45Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'node', 'id': 6835150496, 'lat': 52.5226885, 'lon': 13.3979877, 'tags': {'leisure': 'pitch', 'sport': 'table_tennis', 'wheelchair': 'yes'}}, {'type': 'node', 'id': 6835150497, 'lat': 52.5227083, 'lon': 13.3978939, 'tags': {'leisure': 'pitch', 'sport': 'table_tennis', 'wheelchair': 'yes'}}, {'type': 'node', 'id': 6835150598, 'lat': 52.5229822, 'lon': 13.3965893, 'tags': {'access': 'customers', 'leisure': 'pitch', 'sport': 'table_tennis'}}, {'type': 'node', 'id': 6835150599, 'lat': 52.5229863, 'lon': 13.3964894, 'tags': {'access': 'customers', 'leisure': 'pitch', 'sport': 'table_tennis'}}]}

df = pd.DataFrame(ping_pong["elements"])
df["tags"]

0    {'leisure': 'pitch', 'sport': 'table_tennis', ...
1    {'leisure': 'pitch', 'sport': 'table_tennis', ...
2    {'access': 'customers', 'leisure': 'pitch', 's...
3    {'access': 'customers', 'leisure': 'pitch', 's...
Name: tags, dtype: object

In [56]:
import json
keys = []
nodes = []
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    unique_tags_dict = count_tag_frequency(nodes)
    num_unique_values = {k:len(v) for k, v in unique_tags_dict.items()}
    num_unique_values = {
        k: v
        for k, v in sorted(
            num_unique_values.items(), key=lambda item: item[1], reverse=True
        )
    }

unique_tags_dict_sorted = [unique_tags_dict[k] for k in list(num_unique_values.keys())]

unique_tags_dict

{'highway': ['traffic_signals',
  'crossing',
  'turning_circle',
  'motorway_junction',
  'bus_stop',
  'give_way',
  'mini_roundabout',
  'elevator',
  'street_lamp',
  'passing_place',
  'stop',
  'turning_loop',
  'steps',
  'speed_camera',
  'traffic_sign'],
 'button_operated': ['yes', 'no'],
 'crossing': ['traffic_signals',
  'no',
  'dashes',
  'unmarked',
  'both',
  'yes',
  'uncontrolled',
  'marked',
  'right',
  'zebra',
  'left',
  'informal',
  'surface',
  'pelican'],
 'tactile_paving': ['yes', 'no', 'incorrect'],
 'traffic_signals': ['yes',
  'no',
  'backward',
  'forward',
  'signal',
  'emergency',
  'pedestrian_crossing',
  'tram_priority',
  'on_demand'],
 'TMC': ['Point',
  'negative',
  '9.00',
  '21547',
  '21548',
  '32735',
  '21546',
  '31169',
  '31170',
  '31168',
  '31171',
  'positive',
  '32778',
  '21545',
  '21543',
  '21549',
  '21551',
  '21552',
  '21550',
  'both'],
 'contact': ['+49 30 297 43333',
  'http://www.s-bahn-berlin.de/fahrplanundnetz/bah

In [58]:
def search_dict(d, substring):
    search_words = [s.strip() for s in substring.split(",")]
    print(search_words)
    matches = {}
    for s in search_words:
    # Add key value pairs if a substring appears in either key or value. Value is a list of strings. return only the matching string
        for key, value in d.items():
            if s in key:
                matches[key] = value
            else:
                for v in value:
                    if s in v:
                        if key in matches:
                            matches[key].append(v)
                        else:
                            matches[key] = [v]
    return matches

search_dict(unique_tags_dict, "history, historical")

['history', 'historical']


{'name': ["The German Bundestag's exhibition on parliamentary history"],
 'note': ['history-node of place_of_worship: Herz-Jesu-Kirch'],
 'board_type': ['history'],
 'museum': ['history'],
 'board': ['Myth and historical testimony of the "Führerbunker"']}

In [49]:
len("asdlfkmoadfibno lkansfoinal , lkaosnidfna ")

42

In [ ]:

dict(zip(places_gdf["display_name"], places_gdf["projected_area"]))

In [ ]:
places_gdf[["projected_area", "area_unit"]] = places_gdf.apply(lambda row: gdf_data(row, places_gdf.crs), axis=1)
places_gdf

In [ ]:
for _, row in bounding_boxes.iterrows():
    nodes.append(get_nodes_with_tags_in_bbox(list(row)))
    keys = list(count_tag_frequency(nodes).keys())
keys

In [ ]:
import utm
from pyproj import CRS
import geopandas as gpd
def gdf_data(gdf):
    """Get the area of a polygon
    This method is not directly callable by the LLM"""
    places_dict = {}
    for index, row in gdf.iterrows():
        print(index)
        utm_zone = utm.latlon_to_zone_number(gdf.loc[[index], "lat"].values[0], gdf.loc[[index], "lon"].values[0])
        south = gdf.loc[[index], "lat"].values[0] < 0
        crs = CRS.from_dict({"proj": "utm", "zone": utm_zone, "south": south})
        epsg_code = crs.to_authority()[1]
        unit = list({ai.unit_name for ai in crs.axis_info})[0]
        gdf_projected = gdf.loc[[index],:].to_crs(epsg_code)
        area = gdf_projected.area.values[0]
        places_dict[row["display_name"]] = {"area":area,
                                            "unit":unit}

    return places_dict

In [ ]:
gdf_data(places_gdf)